# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [26]:
from scipy.spatial.distance import pdist, squareform

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [5]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [6]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [80]:
df= data.groupby(["CustomerID", "ProductName"]).agg({"Quantity": "sum"})
df

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1
...                                            ...
98200      Vol Au Vents                         50
           Wasabi Powder                        25
           Wine - Fume Blanc Fetzer             25
           Wine - Hardys Bankside Shiraz        25
           Yogurt - French Vanilla              25

[63628 rows x 1 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [81]:
df_pivot=df.pivot_table(values="Quantity", index="ProductName", columns="CustomerID").fillna(0)
df_pivot

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0
Yoghurt Tubes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0
"Yogurt - Blueberry, 175 Gr",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [82]:
distances = pdist(df_pivot.T, "euclidean")
Q = squareform(distances)

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [109]:
df_top= pd.DataFrame(1/(1+Q), index=df_pivot.columns,columns=df_pivot.columns)
df_top.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


In [125]:
similarities = df_top[464].sort_values(ascending=False)[1:6]
similarities

CustomerID
3317    0.086688
3535    0.085983
2503    0.085638
1008    0.084959
639     0.084626
Name: 464, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [126]:
records =similarities.index.tolist()
records = records[:5]
df_records= data.loc[data['CustomerID'].isin(records)]
df_records.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
201,1008,Leticia,Krueger,4468566,270,Cheese Cloth No 100,1
242,3535,Lana,Peters,2399506,270,Cheese Cloth No 100,1
1038,2503,Kyle,Reeves,182355,417,"Lamb - Pieces, Diced",1
1042,3535,Lana,Peters,4503728,417,"Lamb - Pieces, Diced",1
1362,3535,Lana,Peters,2575012,159,Eggplant - Asian,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [127]:
top_purchase= df_records.groupby("ProductName").agg({"Quantity": "sum"})
top_purchase=top_purchase.sort_values("Quantity", ascending=False)
top_purchase

,Quantity
ProductName,
Butter - Unsalted,4
"Sauce - Gravy, Au Jus, Mix",3
"Pepper - Black, Whole",3
"Peas - Pigeon, Dry",3
Cheese - Cambozola,3
...,...
Flour - Teff,1
Flour - Whole Wheat,1
Foam Cup 6 Oz,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [254]:
df_merge= df_pivot.merge(top_purchase, left_on='ProductName', right_on='ProductName')
df_merge= df_merge[[464, "Quantity"]].sort_values("Quantity", ascending=False)
df_merge=df_merge[df_merge[464] ==0.0]
df_merge[:5].index.tolist()

['Butter - Unsalted',
 'Sauce - Gravy, Au Jus, Mix',
 'Pepper - Black, Whole',
 'Peas - Pigeon, Dry',
 'Bar - Granola Trail Mix Fruit Nut']

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [246]:
empty_dict = {}

In [247]:
lista= list(df_top.columns)

In [248]:
for i in lista:
    similarities = df_top[i].sort_values(ascending=False)[1:6]
    records =similarities.index.tolist()
    records = records[:5]
    df_records= data.loc[data['CustomerID'].isin(records)]
    df_merge= df_pivot.merge(top_purchase, left_on='ProductName', right_on='ProductName')
    df_merge= df_merge[[i, "Quantity"]].sort_values("Quantity", ascending=False)
    df_merge=df_merge[df_merge[i] ==0.0]
    final = df_merge[:5].index.tolist()
    empty_dict[i]=final

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [252]:
results= pd.DataFrame(empty_dict)
results

,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
0,Butter - Unsalted,Butter - Unsalted,Butter - Unsalted,Butter - Unsalted,Butter - Unsalted,Butter - Unsalted,Butter - Unsalted,Butter - Unsalted,Butter - Unsalted,Butter - Unsalted,...,Butter - Unsalted,Butter - Unsalted,Butter - Unsalted,Butter - Unsalted,Butter - Unsalted,Butter - Unsalted,Butter - Unsalted,Butter - Unsalted,"Sauce - Gravy, Au Jus, Mix",Butter - Unsalted
1,"Sauce - Gravy, Au Jus, Mix",Cheese - Cambozola,"Sauce - Gravy, Au Jus, Mix","Sauce - Gravy, Au Jus, Mix","Sauce - Gravy, Au Jus, Mix","Sauce - Gravy, Au Jus, Mix","Pepper - Black, Whole","Pepper - Black, Whole","Sauce - Gravy, Au Jus, Mix","Pepper - Black, Whole",...,"Pepper - Black, Whole","Sauce - Gravy, Au Jus, Mix","Sauce - Gravy, Au Jus, Mix","Sauce - Gravy, Au Jus, Mix","Sauce - Gravy, Au Jus, Mix","Sauce - Gravy, Au Jus, Mix","Sauce - Gravy, Au Jus, Mix","Sauce - Gravy, Au Jus, Mix","Pepper - Black, Whole","Sauce - Gravy, Au Jus, Mix"
2,"Peas - Pigeon, Dry","Veal - Inside, Choice","Pepper - Black, Whole","Pepper - Black, Whole","Pepper - Black, Whole","Pepper - Black, Whole","Peas - Pigeon, Dry",Bar - Granola Trail Mix Fruit Nut,"Pepper - Black, Whole","Peas - Pigeon, Dry",...,"Peas - Pigeon, Dry",Bar - Granola Trail Mix Fruit Nut,"Pepper - Black, Whole","Pepper - Black, Whole","Pepper - Black, Whole","Pepper - Black, Whole","Pepper - Black, Whole","Pepper - Black, Whole","Peas - Pigeon, Dry","Pepper - Black, Whole"
3,Cheese - Cambozola,"Nut - Chestnuts, Whole","Peas - Pigeon, Dry","Peas - Pigeon, Dry","Peas - Pigeon, Dry","Peas - Pigeon, Dry",Cheese - Cambozola,"Nut - Chestnuts, Whole","Peas - Pigeon, Dry",Bar - Granola Trail Mix Fruit Nut,...,Cheese - Cambozola,"Veal - Inside, Choice",Cheese - Cambozola,"Peas - Pigeon, Dry","Peas - Pigeon, Dry",Cheese - Cambozola,"Peas - Pigeon, Dry","Peas - Pigeon, Dry",Cheese - Cambozola,"Peas - Pigeon, Dry"
4,Bar - Granola Trail Mix Fruit Nut,Ketchup - Tomato,Cheese - Cambozola,Cheese - Cambozola,Cheese - Cambozola,Bar - Granola Trail Mix Fruit Nut,Bar - Granola Trail Mix Fruit Nut,Sausage - Breakfast,Cheese - Cambozola,"Veal - Inside, Choice",...,Bar - Granola Trail Mix Fruit Nut,"Nut - Chestnuts, Whole",Bar - Granola Trail Mix Fruit Nut,Cheese - Cambozola,Cheese - Cambozola,Bar - Granola Trail Mix Fruit Nut,Cheese - Cambozola,Cheese - Cambozola,Bar - Granola Trail Mix Fruit Nut,Bar - Granola Trail Mix Fruit Nut


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [268]:
distances = pdist(df_pivot.T, "kulsinski")
Q = squareform(distances)

In [269]:
df_top= pd.DataFrame(1/(1+Q), index=df_pivot.columns,columns=df_pivot.columns)

In [270]:
empty_dict_two = {}

In [271]:
lista= list(df_top.columns)

In [273]:
for i in lista:
    similarities = df_top[i].sort_values(ascending=False)[1:6]
    records =similarities.index.tolist()
    records = records[:5]
    df_records= data.loc[data['CustomerID'].isin(records)]
    df_merge= df_pivot.merge(top_purchase, left_on='ProductName', right_on='ProductName')
    df_merge= df_merge[[i, "Quantity"]].sort_values("Quantity", ascending=False)
    df_merge=df_merge[df_merge[i] ==0.0]
    final = df_merge[:5].index.tolist()
    empty_dict_two[i]=final

In [274]:
results= pd.DataFrame(empty_dict_two)
results

,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
0,Butter - Unsalted,Butter - Unsalted,Butter - Unsalted,Butter - Unsalted,Butter - Unsalted,Butter - Unsalted,Butter - Unsalted,Butter - Unsalted,Butter - Unsalted,Butter - Unsalted,...,Butter - Unsalted,Butter - Unsalted,Butter - Unsalted,Butter - Unsalted,Butter - Unsalted,Butter - Unsalted,Butter - Unsalted,Butter - Unsalted,"Sauce - Gravy, Au Jus, Mix",Butter - Unsalted
1,"Sauce - Gravy, Au Jus, Mix",Cheese - Cambozola,"Sauce - Gravy, Au Jus, Mix","Sauce - Gravy, Au Jus, Mix","Sauce - Gravy, Au Jus, Mix","Sauce - Gravy, Au Jus, Mix","Pepper - Black, Whole","Pepper - Black, Whole","Sauce - Gravy, Au Jus, Mix","Pepper - Black, Whole",...,"Pepper - Black, Whole","Sauce - Gravy, Au Jus, Mix","Sauce - Gravy, Au Jus, Mix","Sauce - Gravy, Au Jus, Mix","Sauce - Gravy, Au Jus, Mix","Sauce - Gravy, Au Jus, Mix","Sauce - Gravy, Au Jus, Mix","Sauce - Gravy, Au Jus, Mix","Pepper - Black, Whole","Sauce - Gravy, Au Jus, Mix"
2,"Peas - Pigeon, Dry","Veal - Inside, Choice","Pepper - Black, Whole","Pepper - Black, Whole","Pepper - Black, Whole","Pepper - Black, Whole","Peas - Pigeon, Dry",Bar - Granola Trail Mix Fruit Nut,"Pepper - Black, Whole","Peas - Pigeon, Dry",...,"Peas - Pigeon, Dry",Bar - Granola Trail Mix Fruit Nut,"Pepper - Black, Whole","Pepper - Black, Whole","Pepper - Black, Whole","Pepper - Black, Whole","Pepper - Black, Whole","Pepper - Black, Whole","Peas - Pigeon, Dry","Pepper - Black, Whole"
3,Cheese - Cambozola,"Nut - Chestnuts, Whole","Peas - Pigeon, Dry","Peas - Pigeon, Dry","Peas - Pigeon, Dry","Peas - Pigeon, Dry",Cheese - Cambozola,"Nut - Chestnuts, Whole","Peas - Pigeon, Dry",Bar - Granola Trail Mix Fruit Nut,...,Cheese - Cambozola,"Veal - Inside, Choice",Cheese - Cambozola,"Peas - Pigeon, Dry","Peas - Pigeon, Dry",Cheese - Cambozola,"Peas - Pigeon, Dry","Peas - Pigeon, Dry",Cheese - Cambozola,"Peas - Pigeon, Dry"
4,Bar - Granola Trail Mix Fruit Nut,Ketchup - Tomato,Cheese - Cambozola,Cheese - Cambozola,Cheese - Cambozola,Bar - Granola Trail Mix Fruit Nut,Bar - Granola Trail Mix Fruit Nut,Sausage - Breakfast,Cheese - Cambozola,"Veal - Inside, Choice",...,Bar - Granola Trail Mix Fruit Nut,"Nut - Chestnuts, Whole",Bar - Granola Trail Mix Fruit Nut,Cheese - Cambozola,Cheese - Cambozola,Bar - Granola Trail Mix Fruit Nut,Cheese - Cambozola,Cheese - Cambozola,Bar - Granola Trail Mix Fruit Nut,Bar - Granola Trail Mix Fruit Nut
